# 代码部分，一个分析聊天记录的demo

In [1]:
import os
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

# 你需要有访问ChatGPT的API key，和网络途径
os.environ["HTTP_PROXY"] = "http://x.x.x.x:yyyy"
os.environ["HTTPS_PROXY"] = "http://x.x.x.x:yyyy"
os.environ["OPENAI_API_KEY"] = 'sk-xxxxxxxxxxxxxxxxx'

E:\envs\openai\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
E:\envs\openai\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
E:\envs\openai\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
template = """
你是一名心理分析师，任务是根据给定的聊天记录和打分指标分析A的心理状态，并根据高分特点与低分特点打出1-10分的分数，并给出得到该分数的理由，若无法分析则得分为'NA'。

{format_instructions}

聊天记录:
\"\"\"
{content}
\"\"\"

指标名：{name}
指标描述：{description}

高分特点: {high}
低分特点: {low}
"""

# 这是用于指定生成格式的
response_schemas = [
    ResponseSchema(name="指标名", description="所分析的指标"),
    ResponseSchema(name="得分", description="1-10的打分，若无法分析则返回'NA'"),
    ResponseSchema(name="原因", description="做出该得分的原因")
]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    template=template,
    input_variables=["content", "name", "description", "high", "low"],
    partial_variables={"format_instructions": format_instructions}
)

In [3]:
wait_analyze = """
A: 睡了嗎
B: 没
A: 快睡
B: [表情]
A: 我睡不著
A: md
A: 好無聊
B: 褪黑素
A: 不要
A: 我只是頭痛
A: 所以睡不著
B: 我相信你不需要睡的
A: 我晚上喝了咖啡
B: 6
A: md
A: 麥當勞的熱飲只有咖啡
A: 不理我
A: 嘻嘻
B: [表情]
A: 嘻嘻
A: 好無聊
A: 嘻嘻
B: 我理你你还怎么睡啊？
A: 有道理
A: 但是妳不是還沒睡嘛
A: 所以就來煩你了
A: 讓你打代碼的生活
A: 增點樂趣
"""

In [4]:
指标 = {
    'N神经质': {
    "说明性描述": "N神经质反映个体情感调节过程，反映个体体验消极情绪的倾向和情绪不稳定性。高神经质个体倾向于有心理压力，不现实的想法、过多的要求和冲动，更容易体验到诸如愤怒、焦虑、抑郁等消极的情绪。他们对外界刺激反应比一般人强烈，对情绪的调节、应对能力比较差，经常处于一种不良的情绪状态下。并且这些人思维、决策、以及有效应对外部压力的能力比较差。相反，神经质维度得分低的人较少烦恼，较少情绪化，比较平静。",
    "低分特点": "非常沉着冷静，不容易惊慌失措。对于强烈的感情，并没有什么反应，即使是在多数人都感到压力的情形下。",
    "高分特点": "容易心烦意乱，甚至是在多数人认为只是满足生活的正常要求下"}
    ,
    'E外向性': {
        "说明性描述": "E外向性来表示人际互动的数量和密度、对刺激的需要以及获得愉悦的能力。这个维度将社会性的、主动的、个人定向的个体和沉默的、严肃的、腼腆的、安静的人作对比。这个方面可由两个品质加以衡量：人际的卷入水平和活力水平。前者评估个体喜欢他人陪伴的程度，而后者反映了个体个人的节奏和活力水平\n外向的人喜欢与人接触，充满活力，经常感受到积极的情绪。他们热情，喜欢运动，喜欢刺激冒险。在一个群体当中，他们非常健谈，自信，喜欢引起别人的注意\n内向的人比较安静，谨慎，不喜欢与外界过多接触。他们不喜欢与人接触不能被解释为害羞或者抑郁，这仅仅是因为比起外向的人，他们不需要那么多的刺激，因此喜欢一个人独处。内向人的这种特点有时会被人误认为是傲慢或者不友好，其实一旦和他接触你经常会发现他是一个非常和善的人",
        "低分特点": "性格内向，缄默和恬静。 喜欢独处，独立活动。社交范围一般局限于少部分亲密的朋友",
        "高分特点": "好交际，待人友好，充满活力，生气勃勃。喜欢投入大量的时间与人相处"}
    ,
    'O开放性': {
        "说明性描述": "O开放性描述一个人的认知风格。对经验的开放性被定义为：为了自身的缘故对经验的前摄（proactive）寻求和对的理解，以及对陌生情境的容忍和探索。这个维度将那些好奇的、新颖的、非传统的以及有创造性的个体与那些传统的、无艺术兴趣的、无分析能力的个体做比较。开放性的人偏爱抽象思维，兴趣广泛。封闭性的人讲求实际，偏爱常规，比较传统和保守。开放性的人适合教授等职业，封闭性的人适合警察、销售、服务性职业等",
        "低分特点": "喜欢在简单和朴素的条件下思考。别人形容你是务实和保守的。",
        "高分特点": "喜欢事物的新奇性，多样性，变革性。具有求知欲、想象力和创造力"}
    ,
    'A宜人性': {
        "说明性描述": "A宜人性是评估个体喜欢与他人一同出现的程度，而宜人性则考察个体对其他人所持的态度，这些态度一方面包括亲近人的、有同情心的、信任他人的、宽大的、心软的，另一方面包括敌对的、愤世嫉俗的、爱摆布人的、复仇心重的、无情的。这里所说的广义的人际定向范围。宜人性代表了“爱”，对合作和人际和谐是否看重。宜人性高的人是善解人意的、友好的、慷慨大方的、乐于助人的，愿意为了别人放弃自己的利益。宜人性高的人对人性持乐观的态度，相信人性本善。宜人性低的人则把自己的利益放在别人的利益之上。本质上，他们不关心别人的利益，因此也不乐意去帮助别人。有时候，他们对别人是非常多疑的，怀疑别人的动机。\n对于某些职位来说，太高的宜人性是没有必要的，尤其是需要强硬和客观判断的场合，例如科学家、评论家和士兵",
        "低分特点": "较少关注他人的需要，而更多地关注自己的需要。人们把你看作强硬的，苛求的，不妥协让步的",
        "高分特点": "对于他人的需要、健康和快乐具有强烈的兴趣。是合意的，具有同情心的，善于与人合作的"}
    ,
    'C严谨性': {
        "说明性描述": "C严谨性指我们控制、管理和调节自身冲动的方式，评估个体在目标导向行为上的组织、坚持和动机。它把可信赖的、讲究的个体和懒散的、马虎的个体作比较。同时反映个体自我控制的程度以及推迟需求满足的能力。冲动并不一定就是坏事，有时候环境要求我们能够快速决策。冲动的个体常被认为是快乐的、有趣的、很好的玩伴。但是冲动的行为常常会给自己带来麻烦，虽然会给个体带来暂时的满足，但却容易产生长期的不良后果，比如攻击他人，吸食毒品等等。冲动的个体一般不会获得很大的成就。谨慎的人容易避免麻烦，能够获得更大的成功。人们一般认为谨慎的人更加聪明和可靠，但是谨慎的人可能是一个完美主义者或者是一个工作狂。极端谨慎的个体让人觉得单调、乏味、缺少生气",
        "低分特点": "喜欢生活在当前的感受中，并且从事现在感觉良好的事情。在从事工作时往往疏忽大意和缺乏组织性",
        "高分特点": "目标明确，并且能够下决心坚持到底。人们认为你值得信赖，而且工作刻苦"}
}

In [5]:
llm = ChatOpenAI(temperature=0)
chain = LLMChain(llm=llm, prompt=prompt)
output = {}
for k, v in 指标.items():
    out = chain.run({"content": wait_analyze,
                     "name": k,
                     "description": v['说明性描述'],
                     "low": v['低分特点'],
                     "high": v['高分特点']
                     })
    output[k] = output_parser.parse(out)
output
# 因为免费的API调用速率有限，一分钟只能调用3次，所以会慢点

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-4bhlTFMeKI6apPhgxvMd0D3I on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 2.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-4bhlTFMeKI6apPhgxvMd0D3I on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit ht

{'N神经质': {'指标名': 'N神经质',
  '得分': '7',
  '原因': 'A在聊天记录中表现出了一些情绪不稳定的迹象，比如睡不着觉、头痛、无聊等，同时也表现出了一些冲动的行为，比如晚上喝咖啡。但是并没有表现出过度的情绪化和焦虑，因此得分为7。'},
 'E外向性': {'指标名': 'E外向性',
  '得分': '3',
  '原因': 'A在聊天中表现出比较内向的特点，不喜欢独处，但也不是很喜欢与人交流，表现出一定的社交恐惧症状。虽然在聊天中有一些互动，但是并没有表现出很强的社交能力和活力。'},
 'O开放性': {'指标名': 'O开放性',
  '得分': '5',
  '原因': '根据聊天记录，A表现出了一定的好奇心和想象力，例如对于咖啡的讨论和对B的煩擾。但是A也表现出了一定的保守和实际，例如对于B提出的褪黑素的建议的拒绝和对于麦当劳咖啡的讨论。因此，A的O开放性得分为5。'},
 'A宜人性': {'指标名': 'A宜人性',
  '得分': '5',
  '原因': '从聊天记录中可以看出，A对B的态度比较友好，但也有一些自我中心的表现，如晚上喝咖啡导致睡不着，还来烦B。因此，可以认为A的宜人性得分在中等水平。'},
 'C严谨性': {'指标名': 'C严谨性',
  '得分': '3',
  '原因': '根据聊天记录可以看出，A在晚上喝了咖啡导致睡不着觉，而B建议A服用褪黑素，但A拒绝了。这表明A缺乏对自己冲动行为的控制和推迟需求满足的能力，同时在工作中也可能存在疏忽大意和缺乏组织性的问题。因此，根据高分特点与低分特点，给出3分的评分。'}}

# 手动实现分析过程，手动在聊天框中输入生成的prompt，然后查看模型的输出

In [6]:
# prompt的生成，这是提交给LLM模型的输入
prompt_format = prompt.format_prompt(
    content=wait_analyze,
    name='N神经质',
    description=指标['N神经质']['说明性描述'],
    low=指标['N神经质']['低分特点'],
    high=指标['N神经质']['高分特点'])
print(prompt_format.text)


你是一名心理分析师，任务是根据给定的聊天记录和打分指标分析A的心理状态，并根据高分特点与低分特点打出1-10分的分数，并给出得到该分数的理由，若无法分析则得分为'NA'。

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "\`\`\`json" and "\`\`\`":

```json
{
	"指标名": string  // 所分析的指标
	"得分": string  // 1-10的打分，若无法分析则返回'NA'
	"原因": string  // 做出该得分的原因
}
```

聊天记录:
"""

A: 睡了嗎
B: 没
A: 快睡
B: [表情]
A: 我睡不著
A: md
A: 好無聊
B: 褪黑素
A: 不要
A: 我只是頭痛
A: 所以睡不著
B: 我相信你不需要睡的
A: 我晚上喝了咖啡
B: 6
A: md
A: 麥當勞的熱飲只有咖啡
A: 不理我
A: 嘻嘻
B: [表情]
A: 嘻嘻
A: 好無聊
A: 嘻嘻
B: 我理你你还怎么睡啊？
A: 有道理
A: 但是妳不是還沒睡嘛
A: 所以就來煩你了
A: 讓你打代碼的生活
A: 增點樂趣

"""

指标名：N神经质
指标描述：N神经质反映个体情感调节过程，反映个体体验消极情绪的倾向和情绪不稳定性。高神经质个体倾向于有心理压力，不现实的想法、过多的要求和冲动，更容易体验到诸如愤怒、焦虑、抑郁等消极的情绪。他们对外界刺激反应比一般人强烈，对情绪的调节、应对能力比较差，经常处于一种不良的情绪状态下。并且这些人思维、决策、以及有效应对外部压力的能力比较差。相反，神经质维度得分低的人较少烦恼，较少情绪化，比较平静。

高分特点: 容易心烦意乱，甚至是在多数人认为只是满足生活的正常要求下
低分特点: 非常沉着冷静，不容易惊慌失措。对于强烈的感情，并没有什么反应，即使是在多数人都感到压力的情形下。



## 生成的prompt如上，可以试着将其原始格式粘贴进ChatGPT的聊天框中，也能得出类似的结果。
## 因为该prompt带有markdown语法，因此渲染后的效果如下：
---

你是一名心理分析师，任务是根据给定的聊天记录和打分指标分析A的心理状态，并根据高分特点与低分特点打出1-10分的分数，并给出得到该分数的理由，若无法分析则得分为'NA'。

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "\`\`\`json" and "\`\`\`":

```json
{
	"指标名": string  // 所分析的指标
	"得分": string  // 1-10的打分，若无法分析则返回'NA'
	"原因": string  // 做出该得分的原因
}
```

聊天记录:
"""

A: 睡了嗎
B: 没
A: 快睡
B: [表情]
A: 我睡不著
A: md
A: 好無聊
B: 褪黑素
A: 不要
A: 我只是頭痛
A: 所以睡不著
B: 我相信你不需要睡的
A: 我晚上喝了咖啡
B: 6
A: md
A: 麥當勞的熱飲只有咖啡
A: 不理我
A: 嘻嘻
B: [表情]
A: 嘻嘻
A: 好無聊
A: 嘻嘻
B: 我理你你还怎么睡啊？
A: 有道理
A: 但是妳不是還沒睡嘛
A: 所以就來煩你了
A: 讓你打代碼的生活
A: 增點樂趣

"""

指标名：N神经质
指标描述：N神经质反映个体情感调节过程，反映个体体验消极情绪的倾向和情绪不稳定性。高神经质个体倾向于有心理压力，不现实的想法、过多的要求和冲动，更容易体验到诸如愤怒、焦虑、抑郁等消极的情绪。他们对外界刺激反应比一般人强烈，对情绪的调节、应对能力比较差，经常处于一种不良的情绪状态下。并且这些人思维、决策、以及有效应对外部压力的能力比较差。相反，神经质维度得分低的人较少烦恼，较少情绪化，比较平静。

高分特点: 容易心烦意乱，甚至是在多数人认为只是满足生活的正常要求下
低分特点: 非常沉着冷静，不容易惊慌失措。对于强烈的感情，并没有什么反应，即使是在多数人都感到压力的情形下。

---
## 对于上述prompt，LLM模型的输出如下，同样是markdown格式的json：
___

```json
{
	"指标名": "N神经质",
	"得分": "7",
	"原因": "A在聊天记录中表现出了一定的情绪不稳定性，比如睡不着觉、头痛、无聊等，同时也表现出了一定的冲动和不现实的想法，比如晚上喝咖啡、不想吃褪黑素等。但是并没有表现出过度的情绪化和强烈的反应，因此得分为7。"
}
```

---
其原始格式如下
